# Strong-form vs Weak-form PINN for the 1D Heat Equation

## Overview

This notebook compares two formulations for solving the 1D heat equation
$u_t = \alpha\, u_{xx}$ using PINNs:

**Strong form** — enforces the PDE residual pointwise at collocation points:
$$\mathcal{L}_f = \frac{1}{N_f}\sum_{i=1}^{N_f}\left(u_t^{(i)} - \alpha\, u_{xx}^{(i)}\right)^2$$

**Weak form** — integrates the residual against compact Gaussian test functions $\psi_k$:
$$\mathcal{L}_f = \sum_k \left|\langle u_t - \alpha\, u_{xx},\, \psi_k \rangle\right|^2$$

The weak form is more forgiving of localised residuals and can improve convergence on problems
where the strong-form residual is hard to minimise pointwise.

**Experiments:**
1. Strong-form forward and inverse problems
2. Weak-form integration method comparison (Simpson-21, GL-15, GL-25)
3. Weak-form test function placement (uniform, random, boundary)

In [ ]:
import torch
import numpy as np
import sys
sys.path.append('..')

from data.heat_data import HeatEquationData
from models.heat_pinn_strategy import StrategicPINN, WeakFormLoss
from training.trainer_strategy import StrategicPINNTrainer
from utils.plotter import plot_solution
from utils.test_functions import generate_compact_gaussians, plot_compact_gaussians

print('Imports successful.')

## Data Generation

In [ ]:
data_gen = HeatEquationData(
    L=1.0, T=1.0, alpha=0.01,
    N_f=10000, N_bc=100, N_ic=200,
    N_sensors=10, N_time_measurements=10,
    noise_level=0.01,
    device='cpu',
    random_seed=42
)

base_data = data_gen.generate_full_dataset()
data_gen.visualize_data(base_data)

In [ ]:
# Strong-form data: includes collocation points
strong_data = {
    'x_f': base_data['x_f'], 't_f': base_data['t_f'],
    'x_bc': base_data['x_bc'], 't_bc': base_data['t_bc'], 'u_bc': base_data['u_bc'],
    'x_ic': base_data['x_ic'], 't_ic': base_data['t_ic'], 'u_ic': base_data['u_ic'],
    'x_m': base_data['x_m'], 't_m': base_data['t_m'], 'u_m': base_data['u_m'],
}


def make_weak_data(test_funcs, test_doms):
    """Assemble a weak-form data dict from the shared base measurements."""
    return {
        'test_funcs': test_funcs,
        'test_doms': test_doms,
        'x_bc': base_data['x_bc'], 't_bc': base_data['t_bc'], 'u_bc': base_data['u_bc'],
        'x_ic': base_data['x_ic'], 't_ic': base_data['t_ic'], 'u_ic': base_data['u_ic'],
        'x_m': base_data['x_m'], 't_m': base_data['t_m'], 'u_m': base_data['u_m'],
    }


def eval_inverse(model, alpha_true=0.01):
    pred = model.get_alpha()
    err  = abs(pred - alpha_true) / alpha_true * 100
    print(f'True α: {alpha_true:.6f}  |  Predicted α: {pred:.6f}  |  Error: {err:.2f}%')
    status = 'SUCCESS' if err < 5 else 'needs longer training'
    print(f'[{status}]')

## Test Functions for the Weak Form

Compact Gaussian test functions are placed across $[0,1]^2$ to form the weak-form residual.
Each function has support radius $r = 0.15$ and the support is clipped at the domain boundary
so it vanishes outside its support domain.

Three centre-placement strategies are available:

| Placement | Description |
|---|---|
| `uniform` | Centres on a regular lattice — even spatial coverage |
| `random`  | Centres sampled uniformly at random |
| `boundary`| Centres concentrated near $x = 0$ and $x = 1$ |

In [ ]:
tf_uniform, td_uniform = generate_compact_gaussians(
    L=1.0, T=1.0, n_funcs=50, support_radius=0.15,
    placement='uniform', min_separation=0.05, smooth=0.0
)

weak_data_uniform = make_weak_data(tf_uniform, td_uniform)

print('Uniform placement:')
plot_compact_gaussians(tf_uniform, td_uniform)

# 1. Strong-form PINN

The strong form minimises the pointwise PDE residual at $N_f = 10\,000$ random collocation
points sampled from $[0,1]^2$.  This is the original PINN formulation (Raissi et al., 2019).

## 1.1 Forward Problem

In [ ]:
strong_fwd = StrategicPINN(
    layers=[2, 50, 50, 50, 50, 1],
    alpha_true=0.01,
    inverse=False
)

trainer_sf = StrategicPINNTrainer(
    model=strong_fwd, data=strong_data,
    learning_rate=1e-3, switch_var=0.1, switch_slope=0.001,
    adaptive_weights=True,
)

trainer_sf.train(epochs=5000, print_every=1000, plot_every=2500)

In [ ]:
plot_solution(model=strong_fwd, data=strong_data, alpha_true=0.01)

## 1.2 Inverse Problem

In [ ]:
strong_inv = StrategicPINN(
    layers=[2, 50, 50, 50, 50, 1],
    inverse=True,
    alpha_init=0.02
)

trainer_si = StrategicPINNTrainer(
    model=strong_inv, data=strong_data,
    learning_rate=1e-3, switch_var=0.1, switch_slope=0.001,
    adaptive_weights=True,
)

trainer_si.train(epochs=5000, print_every=1000, plot_every=2500)

In [ ]:
eval_inverse(strong_inv)
plot_solution(model=strong_inv, data=strong_data, alpha_true=0.01)

# 2. Weak-form PINN

The weak form tests the PDE residual against each test function $\psi_k$ by integrating over
its compact support domain $\Omega_k$.  This avoids requiring the residual to be small at
every collocation point individually, which can regularise training when the solution has
localised structure.

Integration is performed over $\Omega_k$ using either
**Simpson's rule** (uniform grid) or **Gauss–Legendre quadrature** (spectral accuracy for smooth
integrands).

## 2.1 Integration Method Comparison (Forward Problem)

Three integration options on the same uniform test functions:

| Scheme | Points per axis | Approx. convergence order |
|---|---|---|
| Simpson-21 | 21 | 4th order |
| Gauss–Legendre-15 | 15 | 28th order |
| Gauss–Legendre-25 | 25 | 48th order |

In [ ]:
weak_fwd_simp = StrategicPINN(
    layers=[2, 50, 50, 50, 50, 1],
    alpha_true=0.01, inverse=False
)
weak_fwd_simp.set_loss_strategy(
    WeakFormLoss(integration_method='simpson', n_integration_points=21)
)

trainer_wfs = StrategicPINNTrainer(
    model=weak_fwd_simp, data=weak_data_uniform,
    learning_rate=1e-3, switch_var=0.1, switch_slope=0.001,
    adaptive_weights=True,
)

trainer_wfs.train(epochs=5000, print_every=1000, plot_every=2500)
plot_solution(model=weak_fwd_simp, data=weak_data_uniform, alpha_true=0.01)

In [ ]:
weak_fwd_gl15 = StrategicPINN(
    layers=[2, 50, 50, 50, 50, 1],
    alpha_true=0.01, inverse=False
)
weak_fwd_gl15.set_loss_strategy(
    WeakFormLoss(integration_method='gauss_legendre', n_integration_points=15)
)

trainer_wfgl15 = StrategicPINNTrainer(
    model=weak_fwd_gl15, data=weak_data_uniform,
    learning_rate=1e-3, switch_var=0.1, switch_slope=0.001,
    adaptive_weights=True,
)

trainer_wfgl15.train(epochs=5000, print_every=1000, plot_every=2500)
plot_solution(model=weak_fwd_gl15, data=weak_data_uniform, alpha_true=0.01)

In [ ]:
weak_fwd_gl25 = StrategicPINN(
    layers=[2, 50, 50, 50, 50, 1],
    alpha_true=0.01, inverse=False
)
weak_fwd_gl25.set_loss_strategy(
    WeakFormLoss(integration_method='gauss_legendre', n_integration_points=25)
)

trainer_wfgl25 = StrategicPINNTrainer(
    model=weak_fwd_gl25, data=weak_data_uniform,
    learning_rate=1e-3, switch_var=0.1, switch_slope=0.001,
    adaptive_weights=True,
)

trainer_wfgl25.train(epochs=5000, print_every=1000, plot_every=2500)
plot_solution(model=weak_fwd_gl25, data=weak_data_uniform, alpha_true=0.01)

## 2.2 Inverse Problem

Recover $\alpha$ from noisy sensor measurements using the weak form.
Simpson-21 and GL-15 are both tested.

In [ ]:
weak_inv_simp = StrategicPINN(
    layers=[2, 50, 50, 50, 50, 1],
    inverse=True, alpha_init=0.02
)
weak_inv_simp.set_loss_strategy(
    WeakFormLoss(integration_method='simpson', n_integration_points=21)
)

trainer_wis = StrategicPINNTrainer(
    model=weak_inv_simp, data=weak_data_uniform,
    learning_rate=1e-3, switch_var=0.1, switch_slope=0.001,
    adaptive_weights=True,
)

trainer_wis.train(epochs=5000, print_every=1000, plot_every=2500)
eval_inverse(weak_inv_simp)
plot_solution(model=weak_inv_simp, data=weak_data_uniform, alpha_true=0.01)

In [ ]:
weak_inv_gl = StrategicPINN(
    layers=[2, 50, 50, 50, 50, 1],
    inverse=True, alpha_init=0.02
)
weak_inv_gl.set_loss_strategy(
    WeakFormLoss(integration_method='gauss_legendre', n_integration_points=15)
)

trainer_wig = StrategicPINNTrainer(
    model=weak_inv_gl, data=weak_data_uniform,
    learning_rate=1e-3, switch_var=0.1, switch_slope=0.001,
    adaptive_weights=True,
)

trainer_wig.train(epochs=5000, print_every=1000, plot_every=2500)
eval_inverse(weak_inv_gl)
plot_solution(model=weak_inv_gl, data=weak_data_uniform, alpha_true=0.01)

# 3. Test Function Placement Experiments

The choice of test-function centres affects how the weak-form residual covers the domain.
Uniform placement gives even coverage; random placement introduces stochasticity (which can
help escape local minima); boundary placement focuses capacity near the Dirichlet boundaries
where the residual is often largest.

All experiments in this section use Simpson-21 integration.

## 3.1 Random placement

In [ ]:
tf_random, td_random = generate_compact_gaussians(
    L=1.0, T=1.0, n_funcs=50, support_radius=0.15,
    placement='random', min_separation=0.05, smooth=0.0
)

weak_data_random = make_weak_data(tf_random, td_random)
plot_compact_gaussians(tf_random, td_random)

In [ ]:
m_rand_fwd = StrategicPINN(
    layers=[2, 50, 50, 50, 50, 1],
    alpha_true=0.01, inverse=False
)
m_rand_fwd.set_loss_strategy(
    WeakFormLoss(integration_method='simpson', n_integration_points=21)
)

t_rand_fwd = StrategicPINNTrainer(
    model=m_rand_fwd, data=weak_data_random,
    learning_rate=1e-3, switch_var=0.1, switch_slope=0.001,
    adaptive_weights=True,
)

t_rand_fwd.train(epochs=5000, print_every=1000, plot_every=2500)
plot_solution(model=m_rand_fwd, data=weak_data_random, alpha_true=0.01)

In [ ]:
m_rand_inv = StrategicPINN(
    layers=[2, 50, 50, 50, 50, 1],
    inverse=True, alpha_init=0.02
)
m_rand_inv.set_loss_strategy(
    WeakFormLoss(integration_method='simpson', n_integration_points=21)
)

t_rand_inv = StrategicPINNTrainer(
    model=m_rand_inv, data=weak_data_random,
    learning_rate=1e-3, switch_var=0.1, switch_slope=0.001,
    adaptive_weights=True,
)

t_rand_inv.train(epochs=5000, print_every=1000, plot_every=2500)
eval_inverse(m_rand_inv)
plot_solution(model=m_rand_inv, data=weak_data_random, alpha_true=0.01)

## 3.2 Boundary placement

In [ ]:
tf_boundary, td_boundary = generate_compact_gaussians(
    L=1.0, T=1.0, n_funcs=50, support_radius=0.15,
    placement='boundary', min_separation=0.05, smooth=0.0
)

weak_data_boundary = make_weak_data(tf_boundary, td_boundary)
plot_compact_gaussians(tf_boundary, td_boundary)

In [ ]:
m_bnd_fwd = StrategicPINN(
    layers=[2, 50, 50, 50, 50, 1],
    alpha_true=0.01, inverse=False
)
m_bnd_fwd.set_loss_strategy(
    WeakFormLoss(integration_method='simpson', n_integration_points=21)
)

t_bnd_fwd = StrategicPINNTrainer(
    model=m_bnd_fwd, data=weak_data_boundary,
    learning_rate=1e-3, switch_var=0.1, switch_slope=0.001,
    adaptive_weights=True,
)

t_bnd_fwd.train(epochs=5000, print_every=1000, plot_every=2500)
plot_solution(model=m_bnd_fwd, data=weak_data_boundary, alpha_true=0.01)

In [ ]:
m_bnd_inv = StrategicPINN(
    layers=[2, 50, 50, 50, 50, 1],
    inverse=True, alpha_init=0.02
)
m_bnd_inv.set_loss_strategy(
    WeakFormLoss(integration_method='simpson', n_integration_points=21)
)

t_bnd_inv = StrategicPINNTrainer(
    model=m_bnd_inv, data=weak_data_boundary,
    learning_rate=1e-3, switch_var=0.1, switch_slope=0.001,
    adaptive_weights=True,
)

t_bnd_inv.train(epochs=5000, print_every=1000, plot_every=2500)
eval_inverse(m_bnd_inv)
plot_solution(model=m_bnd_inv, data=weak_data_boundary, alpha_true=0.01)

# Conclusions

## Strong form vs Weak form

| Aspect | Strong form | Weak form |
|---|---|---|
| Residual | Pointwise at $N_f$ collocation points | Integrated against $K$ test functions |
| PDE inputs | `x_f`, `t_f` tensors | compact Gaussian functions + domains |
| Memory | $\propto N_f$ | $\propto K \times n_q^2$ |
| Integration | None | Simpson or Gauss–Legendre |
| Noise sensitivity | Higher (pointwise) | Lower (averaged over support) |

## Integration method

All three schemes (Simpson-21, GL-15, GL-25) converge for this smooth PDE.
GL-15 is a good default: spectral accuracy at lower cost than GL-25.

## Test function placement

Uniform placement is the safest default.  Random placement is competitive and can sometimes
converge faster.  Boundary placement is useful when the solution is hardest to fit near
$x = 0$ or $x = 1$.